# Imports and Reading Files

In [1]:
!pip install datasets pandas transformers transformers[torch] scikit-multilearn optuna
!git clone https://github.com/lucasadelino/thesis.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cach

In [2]:
import ast
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset
from google.colab import files
from transformers import AutoModelForSequenceClassification, BertTokenizer, BertForSequenceClassification, EvalPrediction, TrainingArguments, Trainer
from sklearn.metrics import f1_score, roc_auc_score, hamming_loss
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_pickle('thesis/datasets/etpc_reannotated.pkl')

In [4]:
data

,sentence1,sentence2,sentence1_tokenized,sentence2_tokenized,sentence1_scope,sentence2_scope,collapsed_labels,s1_token_labs,s2_token_labs
0,"Amrozi accused his brother, whom he called ""th...","Referring to him as only ""the witness"", Amrozi...","[Amrozi, accused, his, brother, ,, whom, he, c...","[Referring, to, him, as, only, ``, the, witnes...","[3_0, 3_0, 3_0, 3_0, 0_0, 4_0, 0_0, 4_1, 0_0, ...","[4_1, 4_0, 4_0, 0_0, 2_0, 0_0, 0_0, 0_0, 0_0, ...","[0, 2, 3, 4]","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0...."
2,They had published an advertisement on the Int...,"On June 10, the ship's owners had published an...","[They, had, published, an, advertisement, on, ...","[On, June, 10, ,, the, ship, 's, owners, had, ...","[5_0, 0_0, 0_0, 0_0, 0_0, 0_0, 0_0, 0_0, 3_0, ...","[3_0, 3_0, 3_0, 3_0, 5_0, 5_0, 5_0, 5_0, 0_0, ...","[0, 2, 3, 4, 5]","[[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0....","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0...."
4,"The stock rose $2.11, or about 11 percent, to ...",PG&E Corp. shares jumped $1.63 or 8 percent to...,"[The, stock, rose, $, 2.11, ,, or, about, 11, ...","[PG, &, E, Corp., shares, jumped, $, 1.63, or,...","[5_0, 5_0, 4_0, 0_0, 0_0, 8_0, 0_0, 2_0, 0_0, ...","[5_0, 5_0, 5_0, 5_0, 5_0, 4_0, 0_0, 0_0, 0_0, ...","[0, 1, 2, 3, 4, 5, 8]","[[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0....","[[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0...."
5,Revenue in the first quarter of the year dropp...,With the scandal hanging over Stewart's compan...,"[Revenue, in, the, first, quarter, of, the, ye...","[With, the, scandal, hanging, over, Stewart, '...","[0_0, 1_0, 0_0, 0_0, 0_0, 0_0, 0_0, 0_0, 0_0, ...","[1_3, 1_4, 2_0, 2_0, 2_0, 2_0, 2_0, 2_0, 2_0, ...","[0, 1, 2]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1....","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1...."
7,The DVD-CCA then appealed to the state Supreme...,The DVD CCA appealed that decision to the U.S....,"[The, DVD-CCA, then, appealed, to, the, state,...","[The, DVD, CCA, appealed, that, decision, to, ...","[0_0, 7_0, 2_0, 0_0, 0_0, 0_0, 4_0, 0_0, 0_0, ...","[0_0, 7_0, 7_0, 0_0, 1_3, 2_1, 0_0, 0_0, 4_0, ...","[0, 1, 2, 4, 7]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0...."
...,...,...,...,...,...,...,...,...,...
5792,Gehring waived extradition Monday during a hea...,Gehring waived extradition Monday during a hea...,"[Gehring, waived, extradition, Monday, during,...","[Gehring, waived, extradition, Monday, during,...","[0_0, 0_0, 0_0, 0_0, 0_0, 0_0, 0_0, 0_0, 0_0, ...","[0_0, 0_0, 0_0, 0_0, 0_0, 0_0, 0_0, 2_1, 2_1, ...","[0, 2, 3, 8]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0...."
5793,"""I am advised that certain allegations of crim...","""I am advised that certain allegations of crim...","[``, I, am, advised, that, certain, allegation...","[``, I, am, advised, that, certain, allegation...","[0_0, 0_0, 0_0, 0_0, 0_0, 0_0, 0_0, 0_0, 0_0, ...","[0_0, 0_0, 0_0, 0_0, 0_0, 0_0, 0_0, 0_0, 0_0, ...","[0, 2, 3, 8]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0...."
5795,"The deal, approved by both companies' board of...",The acquisition has been approved by both comp...,"[The, deal, ,, approved, by, both, companies, ...","[The, acquisition, has, been, approved, by, bo...","[0_0, 4_0, 2_0, 2_0, 2_0, 2_0, 2_0, 2_0, 2_0, ...","[0_0, 4_0, 0_0, 0_0, 0_0, 0_0, 0_0, 0_0, 0_0, ...","[0, 2, 4]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0...."
5799,"Last week the power station’s US owners, AES C...","The news comes after Drax's American owner, AE...","[Last, week, the, power, station’s, US, owners...","[The, news, comes, after, Drax, 's, American, ...","[3_0, 3_0, 0_0, 5_1, 5_1, 5_0, 6_0, 0_0, 0_0, ...","[2_0, 2_0, 2_0, 2_0, 5_1, 0_0, 5_0, 6_0, 0_0, ...","[0, 2, 3, 5, 6, 7]","[[0.0, 0.0, 1.

In [5]:
data['sentence1_tokenized'] = data['sentence1_tokenized'].apply(lambda x: list(x))
data['sentence2_tokenized'] = data['sentence2_tokenized'].apply(lambda x: list(x))

In [6]:
data['collapsed_labels'] = data['collapsed_labels'].apply(lambda x: [int(i) for i in x])

# Preprocessing

## Train/Test Split

We'll use the collapsed labels here, since iterative stratification has trouble dealing with multi-dimensional arrays.

In [7]:
# Indices selected for each set
# It's easier to just use indices rather than having to deal with sentence pairs here
indices = np.array(data.index.tolist())
indices = np.expand_dims(indices, axis=1)
indices.shape

(3900, 1)

In [8]:
def id_array_to_labels(id_array):
    """Converts an array of indices to a bit array
    e.g. the array [1, 3] is converted to [0, 1, 0, 1]"""
    labels = np.zeros(9)
    labels[id_array] = 1
    return labels.astype(float)

# Example of the output
example = [0, 2, 3, 4, 5]
print(example)
print(id_array_to_labels(example))

[0, 2, 3, 4, 5]
[1. 0. 1. 1. 1. 1. 0. 0. 0.]


In [9]:
# Labels, converted
labels = data['collapsed_labels'].values
labels = [id_array_to_labels(each_list) for each_list in labels]
data['labels'] = labels
labels = np.array(labels)

# Making sure it worked
data['s1_token_labs'][0]

array([[0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

## Calculating descriptive statistics

First, let's vertically stack all of the the labels, from both sentence 1 and sentence 2, into one big array

In [10]:
label_stack = np.concatenate((np.vstack(data['s1_token_labs'].values), np.vstack(data['s2_token_labs'].values)), axis=0)
label_stack.shape

(175199, 8)

Now, let's see how many rows (i.e. how many tokens) have more than one label

In [11]:
row_sums = np.sum(label_stack, axis=1)
label_stack[row_sums == 2].shape

(1743, 8)

In [12]:
1415/175199

0.008076530117181034

Proportion of tokens with 2:
0.009948686921729006

Proportion of tokens with 3:
9.132472217307176e-05

Now, let's get rid of rows where all labels are zero

In [13]:
trimmed_label_stack = label_stack[label_stack.any(axis=1)]
trimmed_label_stack.shape

(49167, 8)

Let's again calculate how many rows have more than one label

In [14]:
trimmed_row_sums = np.sum(trimmed_label_stack, axis=1)
trimmed_label_stack[trimmed_row_sums == 2].shape

(1743, 8)

In [15]:
sent1multi = data['s1_token_labs'].apply(lambda x: np.max(np.sum(x, axis=1)) > 1.0)
sent2multi = data['s2_token_labs'].apply(lambda x: np.max(np.sum(x, axis=1)) > 1.0)

Proportion of sentences with one or more:

In [16]:
sent1_prop = sent1multi.value_counts()[1] / sent1multi.value_counts()[0]
sent2_prop = sent2multi.value_counts()[1] / sent2multi.value_counts()[0]

print(sent1_prop)
print(sent2_prop)
print()

total_prop  = sent1_prop + sent2_prop
print(total_prop)

0.13537117903930132
0.13471050334594123

0.27008168238524255


Proportion of sentences with two or more:

In [17]:
sent1multi = data['s1_token_labs'].apply(lambda x: np.max(np.sum(x, axis=1)) > 2.0)
sent2multi = data['s2_token_labs'].apply(lambda x: np.max(np.sum(x, axis=1)) > 2.0)

In [18]:
sent1_prop = sent1multi.value_counts()[1] / sent1multi.value_counts()[0]
sent2_prop = sent2multi.value_counts()[1] / sent2multi.value_counts()[0]

print(sent1_prop)
print(sent2_prop)
print()

total_prop  = sent1_prop + sent2_prop
print(total_prop)

0.0020554984583761563
0.0017980991523246853

0.0038535976107008417


### Figuring out label distributions

## Collapsing

Let's first determine how to resolve label conflicts:

In [ ]:
map = [
    (np.array([1., 0., 0., 0., 1., 0., 0., 0.]), 4),
    (np.array([0., 1., 0., 0., 0., 1., 0., 0.]), 1),
    (np.array([0., 1., 0., 1., 0., 0., 0., 0.]), 3),
    (np.array([0., 1., 0., 0., 1., 0., 0., 0.]), 4),
    (np.array([0., 1., 0., 0., 0., 0., 0., 1.]), 7),
    (np.array([0., 0., 0., 1., 1., 0., 0., 0.]), 3),
    (np.array([0., 0., 0., 1., 0., 1., 0., 0.]), 3),
    (np.array([0., 0., 0., 1., 0., 0., 1., 0.]), 6),
    (np.array([0., 0., 0., 0., 1., 1., 0., 0.]), 4),
    (np.array([0., 0., 0., 0., 1., 0., 1., 0.]), 6),
    (np.array([0., 0., 0., 0., 1., 0., 0., 1.]), 7),
    (np.array([0., 0., 0., 0., 0., 1., 1., 0.]), 6),
    (np.array([0., 0., 0., 1., 0., 1., 0., 1.]), 5),
    (np.array([0., 0., 0., 0., 0., 1., 0., 1.]), 7),
    (np.array([0., 1., 0., 0., 0., 0., 1., 0.]), 6)
]

In [ ]:
def convert_multi_to_single(label_array):
    # First, check if there's more than one value AND there's a change of order
    # Change of orders will only occur if there's no other substitutions
    if np.sum(label_array) == 0:
        return 0
    if np.sum(label_array) > 1 and label_array[2] == 1:
        label_array[2] = 0
    # Return index if there's only one element
    if np.sum(label_array) == 1:
        return np.argmax(label_array) + 1
    # Lookup the map if there's more than one element
    else:
        for array, value in map:
            if np.array_equal(array, label_array):
                return value + 1

# Example usage:
example = np.array([0., 1., 1., 0., 1., 0., 0., 0.])
convert_multi_to_single(example)

Now, to actually perform the conversion:

In [ ]:
data['s1_token_labs'] = data['s1_token_labs'].apply(lambda x: [convert_multi_to_single(row) for row in x])
data['s2_token_labs'] = data['s2_token_labs'].apply(lambda x: [convert_multi_to_single(row) for row in x])

### Stratification

In [ ]:
# ITERATIVE STRATIFICATION
np.random.seed(3)
# 80/20 split into train and temp sets (temp set will be further split below)
x_train, y_train, x_val_test, y_val_test = iterative_train_test_split(indices, labels, test_size=0.2)
# 50/50 split temp set into validation and test sets
x_val, y_val, x_test, y_test  = iterative_train_test_split(x_val_test, y_val_test, test_size=0.5)

print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(3087, 1)
(414, 1)
(399, 1)


In [ ]:
# Converting indices in x matrices back to 1-d
indices_train = np.squeeze(x_train)
indices_test = np.squeeze(x_test)
indices_val = np.squeeze(x_val)

In [ ]:
train = data.loc[indices_train, :]
test = data.loc[indices_test, :]
val = data.loc[indices_val, :]

## Tokenizing

In [ ]:
#model_type = 'bert-large-uncased'
model_type = 'bert-base-uncased'

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_type)

def tokenize_and_align_labels(example):
    # Tokenize the sentence pair
    tokenized_inputs = tokenizer(example["sentence1_tokenized"], example["sentence2_tokenized"],padding='max_length', max_length=90,
                                 truncation=True, is_split_into_words=True)

    label_array_1 = example['s1_token_labs']  # Label array for the first sentence
    label_array_2 = example['s2_token_labs']  # Label array for the second sentence
    word_ids = tokenized_inputs.word_ids(batch_index=0)

    label_ids = []
    sentence_switch = False  # Flag to indicate when to switch from the first to the second sentence's labels
    previous_word_id = None

    for index, word_id in enumerate(word_ids):
        if word_id is None and not sentence_switch:
            # First [CLS] or [SEP] token encountered
            label_ids.append(-100)
            if index > 0:
                # First [SEP] token encountered
                sentence_switch = True  # Switch to the second sentence's labels
        elif word_id is None:
            # Second [SEP] token or [CLS] token at the end
            label_ids.append(-100)
        else:
            # Normal token, choose appropriate label array
            current_label_array = label_array_2 if sentence_switch else label_array_1
            label_ids.append(current_label_array[word_id])

        previous_word_id = word_id

    tokenized_inputs["labels"] = label_ids

    return tokenized_inputs

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Tokenize sentences and save as new column in dfs
train['tokenized_sentences'] = train.apply(tokenize_and_align_labels, axis=1)
test['tokenized_sentences'] = test.apply(tokenize_and_align_labels, axis=1)
val['tokenized_sentences'] = val.apply(tokenize_and_align_labels, axis=1)

In [ ]:
# Convert tokenized sentences to tensors. Those will be the inputs to our (PyTorch) model
train['inputs'] = train['tokenized_sentences'].apply(lambda x: x.convert_to_tensors('pt'))
test['inputs'] = test['tokenized_sentences'].apply(lambda x: x.convert_to_tensors('pt'))
val['inputs'] = val['tokenized_sentences'].apply(lambda x: x.convert_to_tensors('pt'))

# Training

In [ ]:
from sklearn.metrics import precision_score, recall_score, precision_recall_fscore_support, accuracy_score, f1_score

def compute_metrics(p: EvalPrediction):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    y_pred = [
        p for prediction, label in zip(predictions, labels)
        for p, l in zip(prediction, label) if l != -100
    ]
    y_true = [
        l for label in labels
        for l in label if l != -100
    ]

    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='weighted')
    accuracy = f1_score(y_true, y_pred, average='micro', labels=[1,2,3,4,5,6,7,8])
    inflated_accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true=y_true, y_pred=y_pred, average='weighted')
    recall = recall_score(y_true=y_true, y_pred=y_pred, average='weighted')
    results = {'f1': f1_micro_average,
               'accuracy': accuracy,
               '0accuracy': inflated_accuracy,
               'precision': precision,
               'recall': recall}
    return results

In [ ]:
params = {'learning_rate': 0.00021437583926908025, 'num_train_epochs': 7, 'warmup_steps': 261, 'weight_decay': 0.03953905927366381, 'per_device_train_batch_size': 16}

metric_name = "accuracy"

args = TrainingArguments(
    f"bert-paraop",
    evaluation_strategy = "epoch",
    logging_steps = 10,
    save_strategy = "epoch",
    learning_rate=params['learning_rate'],
    per_device_train_batch_size=params['per_device_train_batch_size'],
    per_device_eval_batch_size=params['per_device_train_batch_size'],
    num_train_epochs=params['num_train_epochs'],
    warmup_steps=params['warmup_steps'],
    weight_decay=params['weight_decay'],
    #optim = 'adamw_torch',
    load_best_model_at_end=True,
    save_total_limit=2,
    metric_for_best_model=metric_name
)

In [ ]:
from transformers import BertForTokenClassification

def model_init():
    return BertForTokenClassification.from_pretrained(model_type, num_labels=9)

In [ ]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
trainer = Trainer(
    model=None,
    args=args,
    train_dataset=train['inputs'].values,
    eval_dataset=val['inputs'].values,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    model_init=model_init
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Hyperparameter Search

In [ ]:
def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 3e-4, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 4, 8),
        "warmup_steps": trial.suggest_int("warmup_steps", 10, 300),
        "weight_decay": trial.suggest_float("weight_decay", 0.01, 0.05, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32, 64, 128]),
    }

In [ ]:
def get_accuracy(input):
    return input['eval_accuracy']

In [ ]:
best_trials = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=optuna_hp_space,
    n_trials=15,
    compute_objective=get_accuracy
)

[I 2024-04-28 20:42:33,065] A new study created in memory with name: no-name-adfa9f03-ba3c-47b5-907d-8ae3b7216b21
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1,Accuracy,0accuracy,Precision,Recall
1,0.872800,0.745739,0.703098,0.284049,0.767236,0.685518,0.767236
2,0.722900,0.664146,0.755487,0.400708,0.789251,0.731954,0.789251
3,0.685900,0.616349,0.777008,0.466127,0.799199,0.768874,0.799199
4,0.644100,0.599474,0.792944,0.498727,0.804449,0.788886,0.804449
5,0.572800,0.592769,0.792471,0.495341,0.808272,0.786913,0.808272
6,0.544500,0.586798,0.796100,0.507876,0.809745,0.791040,0.809745


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

Epoch,Training Loss,Validation Loss,F1,Accuracy,0accuracy,Precision,Recall
1,0.650900,0.600932,0.780548,0.465978,0.804081,0.774927,0.804081
2,0.588600,0.529387,0.805123,0.533891,0.824851,0.812999,0.824851
3,0.451800,0.545477,0.823723,0.579067,0.830148,0.823815,0.830148
4,0.274000,0.612707,0.824042,0.572624,0.827983,0.822419,0.827983
5,0.182300,0.654275,0.828788,0.590069,0.836365,0.825945,0.836365
6,0.134300,0.702531,0.826415,0.587676,0.831391,0.824985,0.831391
7,0.073600,0.757106,0.833758,0.602240,0.838714,0.830620,0.838714


[I 2024-04-28 20:48:52,766] Trial 1 finished with value: 0.6022401184856059 and parameters: {'learning_rate': 0.00021437583926908025, 'num_train_epochs': 7, 'warmup_steps': 261, 'weight_decay': 0.03953905927366381, 'per_device_train_batch_size': 16}. Best is trial 1 with value: 0.6022401184856059.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

[I 2024-04-28 21:09:10,944] Trial 12 pruned. 
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1,Accuracy,0accuracy,Precision,Recall
1,0.749700,0.681428,0.753588,0.404876,0.786303,0.735890,0.786303


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2024-04-28 21:09:40,574] Trial 13 pruned. 
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1,Accuracy,0accuracy,Precision,Recall
1,0.681700,0.599386,0.784802,0.481576,0.804034,0.783118,0.804034


[I 2024-04-28 21:10:19,442] Trial 14 pruned. 


In [ ]:
best_trials

BestRun(run_id='1', objective=0.6022401184856059, hyperparameters={'learning_rate': 0.00021437583926908025, 'num_train_epochs': 7, 'warmup_steps': 261, 'weight_decay': 0.03953905927366381, 'per_device_train_batch_size': 16}, run_summary=None)

### Training with Optimal Hyperparameters

In [ ]:
trainer.train()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1,Accuracy,0accuracy,Precision,Recall
1,0.660200,0.615096,0.783436,0.464000,0.802100,0.788328,0.802100
2,0.571600,0.555714,0.790384,0.503467,0.818404,0.794455,0.818404
3,0.437800,0.570828,0.815805,0.556363,0.823884,0.812235,0.823884
4,0.273600,0.593348,0.824684,0.583392,0.827569,0.825344,0.827569
5,0.171300,0.670793,0.828125,0.590971,0.832266,0.826483,0.832266
6,0.112100,0.728773,0.825602,0.582805,0.831668,0.821517,0.831668
7,0.078600,0.790125,0.828112,0.589491,0.833694,0.824460,0.833694


TrainOutput(global_step=1351, training_loss=0.3809327110124243, metrics={'train_runtime': 146.5142, 'train_samples_per_second': 147.487, 'train_steps_per_second': 9.221, 'total_flos': 992587316340060.0, 'train_loss': 0.3809327110124243, 'epoch': 7.0})

In [ ]:
eval_result = trainer.evaluate()
for item, value in eval_result.items():
    print(f"{item}: {value}")

eval_loss: 0.09071370214223862
eval_f1: 0.4383088869715272
eval_roc_auc: 0.8250157931661872
eval_hamming loss: 0.059964076820338046
eval_runtime: 1.0777
eval_samples_per_second: 384.164
eval_steps_per_second: 12.063
epoch: 12.0


# Testing

In [ ]:
from sklearn.metrics import precision_score, recall_score, precision_recall_fscore_support, accuracy_score

# Defining evaluation metrics
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def test_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    y_pred = [
        p for prediction, label in zip(predictions, labels)
        for p, l in zip(prediction, label) if l != -100
    ]
    y_true = [
        l for label in labels
        for l in label if l != -100
    ]
    labs = [1,2,3,4,5,6,7,8]
    overall_f1_macro = f1_score(y_true=y_true, y_pred=y_pred, average='macro', labels=labs).tolist()
    overall_f1_micro = f1_score(y_true=y_true, y_pred=y_pred, average='micro', labels=labs).tolist()
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average=None, labels=labs).tolist()
    precision_overall = precision_score(y_true=y_true, y_pred=y_pred, average='micro', labels=labs)
    recall_overall = recall_score(y_true=y_true, y_pred=y_pred, average='micro', labels=labs)

    #accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true=y_true, y_pred=y_pred, average=None, labels=labs).tolist()
    recall = recall_score(y_true=y_true, y_pred=y_pred, average=None, labels=labs).tolist()

    results = {'F1': f1_micro_average,
               'Overal F1 Macro': overall_f1_macro,
               'Overall Accuracy': overall_f1_micro,
               #'accuracy': accuracy,
               'Precision': precision,
               'Recall': recall,
               'Precision Overall': precision_overall,
               'Recall Overall': recall_overall}

    return results

In [ ]:
test_result = trainer.predict(test['inputs'].values)

# Print default metrics collected during prediction
for item, value in test_result.metrics.items():
    print(f"{item}: {value}")

predictions = torch.Tensor(test_result.predictions)
labels = torch.Tensor(test_result.label_ids)

# Compute class-wise metrics
#thresholds = [0.15, 0.5, 0.5, 0.5, 0.5, 0.19, 0.5, 0.5]
results = test_metrics((predictions, labels))
df = pd.DataFrame.from_dict(results)
#df.drop(columns = ['f1', 'roc_auc', 'hamming_loss'], inplace=True)
df.index = [['1. Add/Del - Function Word', '2. Add/Del - Content Word', '3. Change of Order',
             '4. Substitution - Synonym', '5. Substitution - Contextual Synonym', '6. Substitution - Morphological',
             '7. Substitution - Spelling and Format', '8. Add/Del - Punctuation']]
df

test_loss: 0.708419919013977
test_f1: 0.8190371055664056
test_accuracy: 0.6173206327012148
test_0accuracy: 0.8250701590271281
test_precision: 0.8166889519987208
test_recall: 0.8250701590271281
test_runtime: 0.8199
test_samples_per_second: 486.632
test_steps_per_second: 30.491


,F1,Overal F1 Macro,Overall Accuracy,Precision,Recall,Precision Overall,Recall Overall
1. Add/Del - Function Word,0.397516,0.570549,0.617321,0.500000,0.329897,0.656994,0.582166
2. Add/Del - Content Word,0.737399,0.570549,0.617321,0.753129,0.722313,0.656994,0.582166
3. Change of Order,0.561488,0.570549,0.617321,0.663830,0.486486,0.656994,0.582166
4. Substitution - Synonym,0.625709,0.570549,0.617321,0.614670,0.637151,0.656994,0.582166
5. Substitution - Contextual Synonym,0.508521,0.570549,0.617321,0.577406,0.454321,0.656994,0.582166
6. Substitution - Morphological,0.557576,0.570549,0.617321,0.666667,0.479167,0.656994,0.582166
7. Substitution - Spelling and Format,0.564530,0.570549,0.617321,0.570707,0.558484,0.656994,0.582166
8. Add/Del - Punctuation,0.611650,0.570549,0.617321,0.750000,0.516393,0.656994,0.582166
